In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

import torch
from transformers import AutoTokenizer, AutoModel

In [3]:
agents_abili = pd.read_csv(r"https://raw.githubusercontent.com/ayanatherate/Valorant-Agents-Recommender/main/data/agents_abilities.csv")

In [4]:
agents_abili

,Name,Role,Biography,Image,1st ability,1st ability description,1st ability example,2nd ability,2nd ability description,2nd ability example,3rd ability,3rd ability description,3rd ability example,Ultimate ability,Ultimate description,Ultimate example
0,KAY/O,initiator,KAY/O is a machine of war built for a single p...,https://images.contentstack.io/v3/assets/bltb6...,FLASH/DRIVE,EQUIP a flash grenade. FIRE to throw. The flas...,https://assets.contentstack.io/v3/assets/bltb6...,ZERO/POINT,EQUIP a suppression blade. FIRE to throw. The ...,https://assets.contentstack.io/v3/assets/bltb6...,FRAG/MENT,EQUIP an explosive fragment. FIRE to throw. Th...,https://assets.contentstack.io/v3/assets/bltb6...,NULL/CMD,INSTANTLY overload with polarized radianite en...,https://assets.contentstack.io/v3/assets/bltb6...
1,Omen,controller,"A phantom of a memory, Omen hunts in the shado...",https://images.contentstack.io/v3/assets/bltb6...,PARANOIA,"INSTANTLY fire a shadow projectile forward, br...",https://assets.contentstack.io/v3/assets/bltb6...,DARK COVER,EQUIP a shadow orb and see its range indicator...,https://assets.contentstack.io/v3/assets/bltb6...,SHROUDED STEP,EQUIP a shadow walk ability and see its range ...,https://assets.contentstack.io/v3/assets/bltb6...,FROM THE SHADOWS,EQUIP a tactical map. FIRE to begin teleportin...,https://assets.contentstack.io/v3/assets/bltb6...
2,Sova,initiator,Born from the eternal winter of Russia's tundr...,https://images.contentstack.io/v3/assets/bltb6...,SHOCK BOLT,EQUIP a bow with a shock bolt. FIRE to send th...,https://assets.contentstack.io/v3/assets/bltb6...,RECON BOLT,EQUIP a bow with a recon bolt. FIRE to send th...,https://assets.contentstack.io/v3/assets/bltb6...,OWL DRONE,EQUIP an owl drone. FIRE to deploy and take co...,https://assets.contentstack.io/v3/assets/bltb6...,HUNTER’S FURY,EQUIP a bow with three long-range wall-piercin...,https://assets.contentstack.io/v3/assets/bltb6...
3,Viper,controller,"The American chemist, Viper deploys an array o...",https://images.contentstack.io/v3/assets/bltb6...,POISON CLOUD,EQUIP a gas emitter. FIRE to throw the emitter...,https://assets.contentstack.io/v3/assets/bltb6...,TOXIC SCREEN,EQUIP a gas emitter launcher. FIRE to deploy a...,https://assets.contentstack.io/v3/assets/bltb6...,SNAKE BITE,EQUIP a chemical launcher. FIRE to launch a ca...,https://assets.contentstack.io/v3/assets/bltb6...,VIPER’S PIT,EQUIP a chemical sprayer. FIRE to spray a chem...,https://assets.contentstack.io/v3/assets/bltb6...
4,Yoru,duelist,"Japanese native, Yoru, rips holes straight thr...",https://images.contentstack.io/v3/assets/bltb6...,BLINDSIDE,EQUIP to rip an unstable dimensional fragment ...,https://assets.contentstack.io/v3/assets/bltb6...,GATECRASH,EQUIP a rift tether FIRE to send the tether fo...,https://assets.contentstack.io/v3/assets/bltb6...,FAKEOUT,EQUIP an echo that transforms into a mirror im...,https://assets.contentstack.io/v3/assets/bltb6...,DIMENSIONAL DRIFT,EQUIP a mask that can see between dimensions. ...,https://assets.contentstack.io/v3/assets/bltb6...
5,Raze,duelist,Raze explodes out of Brazil with her big perso...,https://images.contentstack.io/v3/assets/bltb6...,BLAST PACK,INSTANTLY throw a Blast Pack that will stick t...,https://assets.contentstack.io/v3/assets/bltb6...,PAINT SHELLS,EQUIP a cluster grenade. FIRE to throw the gre...,https://assets.contentstack.io/v3/assets/bltb6...,BOOM BOT,"EQUIP a Boom Bot. FIRE will deploy the bot, ca...",https://assets.contentstack.io/v3/assets/bltb6...,SHOWSTOPPER,EQUIP a rocket launcher. FIRE shoots a rocket ...,https://assets.contentstack.io/v3/assets/bltb6...
6,Skye,initiator,"Hailing from Australia, Skye and her band of b...",https://images.contentstack.io/v3/assets/bltb6...,TRAILBLAZER,EQUIP a Tasmanian tiger trinket. FIRE to send ...,https://assets.contentstack.io/v3/assets/bltb6...,GUIDING LIGHT,EQUIP a hawk trinket. FIRE to send it forward....,https://assets.contentstack.io/v3/assets/bltb6...,REGROWTH,"EQUI

In [72]:
class VisualizeTextEmbeddings:

  """
  given an array of text paragraphs, embeds using bert-base-uncased,
      reduces to 2D using TSNE for visualizaion using Plotly
  """

  def __init__(self):
        self.tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model=AutoModel.from_pretrained("bert-base-uncased")
        self.tsne=TSNE(n_components=2, random_state=42,perplexity=15)

  def get_distilbert_embeddings(self, texts):
      inputs = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
      with torch.no_grad():
        outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
      return embeddings.numpy()

  def get_visualize_tsne_2Dim(self,texts,label=None,color=None):
      bert_text_embs=self.get_distilbert_embeddings(texts)
      emb_2d= self.tsne.fit_transform(bert_text_embs)
      fig=px.scatter(x=emb_2d[:,0], y=emb_2d[:,-1],text=label,color=color,size=[35 for i in range(len(emb_2d))])
      fig.show()

emb_model = VisualizeTextEmbeddings()

**Visualizing descriptions of each agents and their abilities in shared vector space**

Agents placed closed to each other in the shared vector space can be termed
as "similar" in terms of their characteristics and abilities (although TSNE projection isnt definitive)

In [69]:
emb_model.get_visualize_tsne_2Dim(agents_abili['Biography'].values.tolist(), label=agents_abili['Name'].values.tolist(), color=agents_abili['Role'])

In [74]:
emb_model.get_visualize_tsne_2Dim(agents_abili['2nd ability description'].values.tolist(),label=agents_abili['Name'].values.tolist(),color=agents_abili['Role'])

Concatenation of all the abilities of the Agents Visualized

In [61]:
agents_abili['Concat_abilities']=agents_abili['1st ability description']+agents_abili['2nd ability description']+ agents_abili['3rd ability description'] + agents_abili['Ultimate description']

In [75]:
emb_model.get_visualize_tsne_2Dim(agents_abili['Concat_abilities'].values.tolist(),label=agents_abili['Name'].values.tolist(),color=agents_abili['Role'])